In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tashaphyne.stemming import ArabicLightStemmer


In [3]:
# Load and preprocess data
data = pd.read_csv('C:\\Users\\Musae\\Documents\\GitHub-REPOs\\NLP-Project\\data\\ar_reviews_100k.tsv', sep='\t')
data = data[data['label'] != 'Mixed']

In [4]:
# Initialize stemmer
ArListem = ArabicLightStemmer()

# Load stopwords and punctuations
arabic_stopwords = set(stopwords.words('arabic'))

In [5]:
# Preprocess text function
def preprocess_text(text):
    tokens = word_tokenize(text)
    cleaned_tokens = [word for word in tokens if word not in arabic_stopwords]
    stemmed_tokens = [ArListem.light_stem(word) for word in cleaned_tokens]
    return ' '.join(stemmed_tokens)

# Apply preprocessing
data['cleaned_text'] = data['text'].apply(preprocess_text)

In [6]:
# Define keywords for classification
positive_keywords = {'ممتاز', 'جيد', 'رائع', 'سعيد', 'لذيذ', 'مبهج', 'فرح', 'استثنائي', 'جميل', 'محبب', 'ممتع',
                     'مذهل', 'مريح', 'راض', 'أحب', 'استمتع', 'مفاجأة', 'مميز', 'لطيف', 'مرح', 'معجزة', 'ملهم', 'أسعد',
                     'خيالي', 'مذهل', 'فريد', 'هائل', 'راقي', 'أنيق', 'بهجة', 'مفيد', 'قيمة', 'بسيط', 'ناجح', 'موفق',
                     'مشجع', 'ما شاء الله', 'ظريف', 'محبوب', "اوف",'مبهر', 'إيجابي', 'تفاؤل', 'إعجاب', 'ممتن', 'شجاع', 'آمن', 'مثالي'}

negative_keywords = {'سيء', 'مخيب', 'حزين', 'مؤلم', 'كريه', 'قبيح', 'فشل', 'محبط', 'بشع', 'فظيع', 'مزعج', 'مروع',
                     'أسوأ', 'كره', 'كارثة', 'رعب', 'كئيب', 'مزعزع', 'اكتئاب', 'بائس', 'معقد', 'إحباط', 'تعب', 'مضجر',
                     'ممل', 'فضيحة', 'سلبي', 'كاذب', 'احتيال', 'احراج', 'بشع', 'تعيس', 'مستاء', 'مشؤوم',
                     'عداء', 'مزري', 'عنيف', 'ضعيف', 'متشائم', 'غاضب', 'اهب', 'مروع'}


## Rule-based classification 


In [7]:
def classify_review(text):
    words = set(text.split())
    if words & negative_keywords:
        return "Negative"
    elif words & positive_keywords:
        return "Positive"
    return "Unclassified"  # if neither


In [8]:
# Apply classification
data['predicted_label'] = data['cleaned_text'].apply(classify_review)

# Evaluate classifier
accuracy = (data['predicted_label'] == data['label']).mean()
print(f"\nAccuracy: {accuracy:.2%}")



Accuracy: 43.81%


In [20]:
new_text = "هذا المطعم مليان فيران"

cleaned_new_text = preprocess_text(new_text)
classification_result = classify_review(cleaned_new_text)
print(f"Classification Result: {classification_result}")

Classification Result: Unclassified
